In [1]:
import datasets
import pandas as pd

from datasets import load_dataset

In [2]:
cola_dataset = load_dataset('glue', 'cola')

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


OSError: Cannot find data file. 
Original error:
[WinError 126] The specified module could not be found. Error loading "E:\Anaconda\lib\site-packages\torch\lib\asmjit.dll" or one of its dependencies.

In [ ]:
cola_dataset

In [ ]:
train_dataset = cola_dataset['train']
val_dataset = cola_dataset['validation']
test_dataset = cola_dataset['test']

In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

In [ ]:
train_dataset[0]

In [ ]:
test_dataset[0]

In [ ]:
val_dataset[0]

In [ ]:
train_dataset.features

In [ ]:
train_dataset.filter(lambda example: example['label'] == train_dataset.features['label'].str2int('acceptable'))[:5]

In [ ]:
train_dataset.filter(lambda example: example['label'] == train_dataset.features['label'].str2int('unacceptable'))[:5]

Tokenizing

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

In [ ]:
tokenizer

In [ ]:
print(train_dataset[0]['sentence'])
tokenizer(train_dataset[0]['sentence'])

In [ ]:
tokenizer.decode(tokenizer(train_dataset[0]['sentence']))['input_ids']

In [ ]:
def encode(examples):
    return tokenizer(
            examples['sentence'],
        truncation=True, 
        padding='max_length',
        max_length = 512
    )

In [ ]:
train_dataset = train_dataset.map(encode, batched = True)

Formatting

In [ ]:
import torch 

In [ ]:
train_dataset.set_format(type='torch', columns= ['input_ids', 'attention_mask', 'label'])

Data Loader


In [ ]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 32)

In [ ]:
next(iter(dataloader))

In [ ]:
for batch in dataloader:
    print(batch['input_ids'].shape, batch['attention_mask'].shape, batch['label'].shape)

data.py

In [ ]:
import torch
import datasets
import pytorch_lightning as pl
from datasets import load_dataset
from transformers import AutoTokenizer

In [ ]:
class DataModule(pl.LightningDataModule):
    def __init__(self, model_name, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
    def prepare_data(self):
        cola_dataset = load_dataset("glue", 'cola')
        self.train_data = cola_dataset['train']
        self.val_data = cola_dataset['validation']
        
    def tokenize_data(self, example):
        self.tokenizer(
            example['sentence'],
            truncation = True,
            padding = 'max_length',
            max_length = 512
        )
        
    def setup(self, stage = None):
        
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset, batch_size = self.batch_size, shuffle=True
        )
 
    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset, batch_Size = self.batch_size, shuffle=True
        )
    
if __name__=='__main__':
    data_model = DataModule()
    data_model.prepare_data()
    data_model.setup()
    print(next(iter(data_model.train_dataloader()))["input_ids"].shape)

In [ ]:
import torch
import torch.nn asn
import pytorch_lightning as pl
import torch.nn.functional as F
from transformers import AutoModel
from skleean.metrics import accuracy_score

In [ ]:
class ColaModel(pl.LightningModule):
    def __init__(self, model_name='', lr = 1e-2):
        super(ColaModel).__init__()
        self.save_hyperparameters()
        
        self.bert = AutoModel.from_pretrained(model_name)
        
        self.W = nn.Linear(self.bert.config.hiden_size, 2)
        self.num_classes = 2
        
    def forward(self, input_ids, attention_masks):
        outputs = self.bert(input_ids = input_ids, attention_mask=attention_mask)
        
        h_cls = output.last_hidden_stat[:, 0]
        logits = self.W(h_cls)
        return logits
    
    def training_step(self, batch, batch_idx):
        logits = self.forward(batch["input_ids"], batch['attention_mas'])
        loss = F.cross_entropy(logits, batch['label'])
        self.

train.py

In [ ]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from data import DataModel
from model import ColaModel

def main():
    cola_data = DataModule()
    cola_model = ColaModel()
    
    checkpoints_callback = ModelCheckpoint(
        dirpath='./models', monitor='val_loss', mode='min'
    )
    
    early_stopping_callback = EarlyStopping(
        monitor='val_loss', patience=3, verbose=True, model='min'
    )
    
    
    
    
if __name__=='__main__':
    main()

inference.py

In [ ]:
import torch 
from model import ColaModel
from data import DataModule

class ColaPredictor:
    def __init__(self, model_path):
        self.model_path = model_path
        self.model = ColaModel.load_from_checkpoint(model_path)
        self.model.eval()
        self.model.freeze()
        self.processor = DataModule()
        self.softmax = torch.nn.Softmax(dim=0)
        self.labels = ['acceptable', 'unacceptable']
        
    def predict(self, text):
        inference_sample = {
            "sentence":text
        }
        
        processed = self.processor.tokenize_data(inference_sample)
        logits = self.model(
            torch.tensor([processed['input_ids']]),
            torch.tensor([processed['attention_mask']]),
        )
        scores = self.softmax(logits[0]).tolist()
        predictions = []
        for score, label in zip